In [ ]:
import pandas as pd

In [ ]:
## load data
epi_dir = '/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/'
cluster_file = f"{epi_dir}data/2021-01-26_cluster_isolate_summary_table.xlsx"

In [ ]:
df = pd.read_excel(cluster_file, index_col=0)